# PCSkinAndBlood

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'pcskinandblood'
data_type = 'methylation'
model_class = 'PCLinearModel'
species = 'Homo sapiens'
year = 2022
approved_by_author = '⌛'
citation = "Higgins-Chen, Albert T., et al. \"A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking.\" Nature aging 2.7 (2022): 644-661."
doi = "https://doi.org/10.1038/s43587-022-00248-2"
notes = None

## Download necessary data

In [3]:
#download PCClock Rdata file from https://yale.app.box.com/s/kq0b0a7lxckxjvaz7x5n4keaug7tewry
logger = pya.logger.Logger()
url = "https://pyaging.s3.amazonaws.com/supporting_files/CalcAllPCClocks.RData"
dir = "."
pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./CalcAllPCClocks.RData
|-----------> in progress: 100.0000%


#### Download from R package

Let's first write an R script:

In [4]:
%%writefile download.r

library(dplyr)
library(tibble)
library(tidyr)
library(jsonlite)

load(file = "CalcAllPCClocks.RData")

print(ls(all.names = TRUE))

write_json(CalcPCHorvath2, "CalcPCHorvath2.json", digits = 10)
write_json(CpGs, "PCHorvath2CpGs.json")
write_json(imputeMissingCpGs, "PCHorvath2ReferenceCpGBetas.json", digits = 10)

Writing download.r


Let's run the R script. For this, R needs to be installed.

In [5]:
os.system("Rscript download.r")

0

## Load data

#### From JSON file

In [6]:
with open('PCHorvath2CpGs.json', 'r') as f:
    features = json.load(f)

with open('CalcPCHorvath2.json', 'r') as f:
    weights_dict = json.load(f)

with open('PCHorvath2ReferenceCpGBetas.json', 'r') as f:
    reference_feature_values = json.load(f)

## Extract features and weights

First, let's extract the features and weights:

In [7]:
weights = torch.tensor(weights_dict['model']).float()
intercept = torch.tensor(weights_dict['intercept']).float()
center = torch.tensor(weights_dict['center']).float()
rotation = torch.tensor(weights_dict['rotation']).float()

## Load weights into pyaging model

#### PC linear model

In [8]:
model = pya.models.PCLinearModel(input_dim=len(features), pc_dim=rotation.shape[1])

model.center.data = center
model.rotation.data = rotation
model.linear.weight.data = weights.unsqueeze(0)
model.linear.bias.data = intercept

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [9]:
reference_feature_values = dict(zip(features, reference_feature_values))

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [10]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [11]:
def postprocessing_function(x):
    """
    Applies an anti-logarithmic linear transformation to a PyTorch tensor.
    """
    adult_age=20
    
    if x < 0:
        # Exponential transformation for negative values
        age_tensor = (1 + adult_age) * math.exp(x) - 1
    else:
        # Linear transformation for non-negative values
        age_tensor = (1 + adult_age) * x + adult_age
    return age_tensor
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = None

postprocessing = {
    'name': 'anti_log_linear',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [12]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [13]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [14]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [15]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [16]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [17]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [18]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: PCHorvath2ReferenceCpGBetas.json
Deleted file: CalcAllPCClocks.RData
Deleted file: PCHorvath2CpGs.json
Deleted file: download.r
Deleted file: CalcPCHorvath2.json
Deleted folder: .ipynb_checkpoints
